###  Activity 2.0: Loop Ordering and False Sharing

This activity reinforces the concept of reduction and the caching principles taught in the lecture on Cilk on Sep. 18. It is recommended that you run this on the CS machines `gradx.cs.jhu.edu` or `ugradx.cs.jhu.edu`.  The results make sense here.  It is OK to run this on any machine that has at least 4 cores.  If you run on different machine, you may end up with slightly different results. It is OK if your results don't track exactly with the expected findings. On my M1 laptop the results get confusing.

**Due date**: Thursday September 28, 2023, 9:00 pm EDT.

**Instructions for Submission**: Submit via Gradescope.

### The Program

The is a nested loop program that counts the number of occurences of a list of tokens in an array of elements. This is a common computing pattern in data analytics. This could be used to count the number of messages sent in a network from a set of sources.

There are two serial versions of the program.  These are:
  * `countTokensElementsFirst`: loop over the larger elements array in the outer loop and the smaller tokens array in the inner loop.
  * `countTokensTokensFirst`: loop over the larger elements array in the outer loop and the smaller tokens array in the inner loop.
  
This is not a 2-d dimensional data structure like our previous examples. It is 2 separate arrays.

#### Programming


Complete the *TODO* instructions in [activities/tokens_omp.cpp].

1. Add `parallel for` directives to functions:
   * `omp_countTokensElementsFirst`
   * `omp_countTokensTokensFirst`
2. Add `parallel for` and `reduction` directive for the array `token_counts` for:
    * `omp_countTokensElementsFirst_reduce`
    * `omp_countTokensTokensFirst_reduce`
  
The array reduction clause was added to OpenMP and requires one to specify the length of the array.  A simple example is provided in https://dvalters.github.io/optimisation/code/2016/11/06/OpenMP-array_reduction.html.

3. Unroll the loop 8 times in `unroll_omp_countTokensElementsFirst_reduce`. You may assume the the tokens array is evenly divisible by 8.

On the `gradx.cs.jhu.edu` machine after I added this code, I got the timing results
```
Tokens First time: 8.07097 seconds
Elements First time: 6.93468 seconds
OMP Tokens First time: 2.10465 seconds
OMP Elements First time: 1.78919 seconds
OMP Tokens First Reduce time: 1.99353 seconds
OMP Elements First Reduce time.: 1.78073 seconds
Unroll OMP Elements First Reduce time.: 0.926184 seconds
```
building with the command line
```
g++ -O0 -fopenmp tokens_omp.cpp
```
Compiling with `-O0` turns off all compiler optimizations to prevent the compiler from making unknown optimizations that would confound our results.

#### Questions

Provide brief but complete answers to the following questions in the following cell.

1. Why is it more efficient to iterate over the `tokens` in the inner loop? 
(_Note_: Access to both arrays is sequential. This is a question of cache capacity and cache misses.)
2. Of the functions `omp_countTokensElementsFirst` and `omp_countTokensTokensFirst`
    1. Which function performs (unsafe) sharing in the tokens array?
    2. Which function assigns different elements of the token array to different threads?
3. For the function that assigns different tokens to different threads, how does false sharing arise? Be specific about the memory access pattern or include a drawing.
4. For the unrolled loop, why is it more efficient? What computations are avoided?

#### Answers

1. It is more efficient to iterate over the tokens in the inner loop since the array of tokens is much smaller than the the array of elements. If we iterate over the elements in the inner loop, we will incur several cache misses when iterating over the large array of elements, which will then be repeated for each iteration of the outer loop. If there are $n$ cache misses when iterating sequentially over the elements array and $k$ tokens, we will encounter $nk$ cache misses from this loop order. On the other hand, if we iterate over the tokens in the inner loop, the cache misses from iterating over the large elements array will only happen once since it is in the outer loop resulting only, in $n$ cache mises from the elements array.
2. ->
    1. The function omp_countTokensElementsFirst performs unsafe sharing in the tokens array. Since we parallelized the outer for loop, this function assigns different elements of the elements array to different threads. However, different elements of the elements array may match the same token. Thus, we would have different threads attempting to increment the same variable in memory in the tokens array, creating a race condition. 
    2. The function omp_countTokensTokensFirst assigns different elements of the token array to different threads since the outer loop iterates over the tokens. Since that is the loop we parallelize, each thread will get assigned a different iteration of the loop which corresponds to a different token.
3. Each thread will be updating their own tokens variable wihin the tokens array. However, since the tokens array is small, the memory location each thread is accessing within the tokens array will likely lie on the same memory line. As a result, every time one thread updates their local token variable, the entire memory line will be marked as dirty. When another thread tries to update or read their local token variable, it has to update its own cache since the first thread modified the line of memory. This pattern where each thread has their own local variable but all of them are on one memory line resulting in performance loss due to the line of memory constantly being dirty from the operations of other threads is false sharing.
4. The unrolled loop is more efficient as it does 8 loop calculations of the original loop in one loop of the unrolled loop. This means the unrolled loop has 1/8th the amount of conditional checks and branching instructions that must be performed at the start of every loop.

Output:
```
Tokens First time: 1.51301 seconds
Elements First time: 1.6647 seconds
OMP Tokens First time: 0.411279 seconds
OMP Elements First time: 0.434862 seconds
OMP Tokens First Reduce time: 0.389333 seconds
OMP Elements First Reduce time.: 0.427775 seconds
Unroll OMP Elements First Reduce time.: 0.342174 seconds
```
Updated C++ code:
```c++
#include <iostream>
#include <chrono>
#include <omp.h>

// initialize elements to random integer values 0 to range-1
void initElements (unsigned int range, unsigned int num_els, unsigned int* elements) {
    for (int i=0; i<num_els; i++)  {
        elements[i] = rand() % range;
    }
}

// initialize tokens to search.  again 0 to range-1
// note, we should probably enforce that tokens are unique. not important for performance.
void initTokens (int range, int num_toks, unsigned int* tokens) {
    for (int i=0; i<num_toks; i++)  {
        tokens[i] = rand() % range;
    }
}

// initialize all token counts to zero
void initCounts (int num_toks, unsigned int* token_counts) {
    for (int i=0; i<num_toks; i++)  {
        token_counts[i] = 0;
    }
}

// count the number of appearances of each token in the data
void countTokensElementsFirst (unsigned int num_els, unsigned int num_tokens,
                  unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    /* for all elements in the array */
    for (int el=0; el<num_els; el++) {
        /* for all tokens in the list */
        for (int tok=0; tok<num_tokens; tok++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}

// count the number of appearances of each token in the data
void countTokensTokensFirst (unsigned int num_els, unsigned int num_tokens,
                             unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    /* for all tokens in the list */
    for (int tok=0; tok<num_tokens; tok++) {
        /* for all elements in the array */
        for (int el=0; el<num_els; el++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}

// count the number of appearances of each token in the data
void omp_countTokensElementsFirst (unsigned int num_els, unsigned int num_tokens,
                  unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    //TODO parallel for
    /* for all elements in the array */
    #pragma omp parallel for
    for (int el=0; el<num_els; el++) {
        /* for all tokens in the list */
        for (int tok=0; tok<num_tokens; tok++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}

// count the number of appearances of each token in the data
void omp_countTokensTokensFirst (unsigned int num_els, unsigned int num_tokens,
                             unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    //TODO parallel for
    /* for all tokens in the list */
    #pragma omp parallel for
    for (int tok=0; tok<num_tokens; tok++) {
        /* for all elements in the array */
        for (int el=0; el<num_els; el++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}

// elements first with reduction
void omp_countTokensElementsFirst_reduce (unsigned int num_els, unsigned int num_tokens,
                             unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    //TODO parallel for reduction
    /* for all elements in the array */
    #pragma omp parallel for reduction(+:token_counts[:num_tokens])
    for (int el=0; el<num_els; el++) {
        /* for all tokens in the list */
        for (int tok=0; tok<num_tokens; tok++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}

// tokens first with reduction
void omp_countTokensTokensFirst_reduce (unsigned int num_els, unsigned int num_tokens,
                             unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    //TODO parallel for reduction
    /* for all tokens in the list */
    #pragma omp parallel for reduction(+:token_counts[:num_tokens])
    for (int tok=0; tok<num_tokens; tok++) {
        /* for all elements in the array */
        for (int el=0; el<num_els; el++) {
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
        }
    }
}


// unroll tokens elements first with reduction
void unroll_omp_countTokensElementsFirst_reduce (unsigned int num_els, unsigned int num_tokens,
                             unsigned int* elements, unsigned int* tokens, unsigned int* token_counts) {

    //TODO parallel for reduction 
    /* for all elements in the array */
    #pragma omp parallel for reduction(+:token_counts[:num_tokens])
    for (int el=0; el<num_els; el++) {
        /* for all tokens in the list */
        for (int tok=0; tok<num_tokens; tok+=8) {
            //TODO unroll loop 8 times
            /* update the count for the token */
            if (elements[el] == tokens[tok]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+1]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+2]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+3]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+4]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+5]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+6]) {
                token_counts[tok]++;
            }
            else if (elements[el] == tokens[tok+7]) {
                token_counts[tok]++;
            }
        }
    }
}


int main() {

    const unsigned int range = 4096;
    const unsigned int num_tokens = 128;
    const unsigned int num_elements = 4096*256;
    const unsigned int loop_iterations = 16;

    unsigned int tokens[num_tokens];
    unsigned int elements[num_elements];
    unsigned int token_counts[num_tokens];

    initElements(range, num_elements, elements);
    initTokens(range, num_tokens, tokens);
    initCounts(num_tokens, token_counts);

    omp_set_num_threads(4);

    // run once to warm the cache
    countTokensTokensFirst(num_elements, num_tokens, elements, tokens, token_counts);

    // countTokensTokensFirst
    // Start the timer
    auto start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        countTokensTokensFirst(num_elements, num_tokens, elements, tokens, token_counts);
    }
    // Stop the timer
    auto end = std::chrono::high_resolution_clock::now();
    // Calculate the duration
    std::chrono::duration<double> duration = end - start;
    // Print the duration in seconds
    std::cout << "Tokens First time: " << duration.count() << " seconds" << std::endl;

    // reset counts only works right if running one loop_iteration
    initCounts(num_tokens, token_counts);

    // run once to warm the cache
    countTokensElementsFirst(num_elements, num_tokens, elements, tokens, token_counts);
  

    // countTokensElementsFirst
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        countTokensElementsFirst(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "Elements First time: " << duration.count() << " seconds" << std::endl;

    // reset counts only works right if running one loop_iteration
    initCounts(num_tokens, token_counts);
    	
    
    // omp_countTokensTokensFirst
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        omp_countTokensTokensFirst(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "OMP Tokens First time: " << duration.count() << " seconds" << std::endl;

    // reset counts only works right if running one loop_iteration
    initCounts(num_tokens, token_counts);

    
    // omp_countTokensElementsFirst
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        omp_countTokensElementsFirst(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "OMP Elements First time: " << duration.count() << " seconds" << std::endl;

    // reset counts only works right if running one loop_iteration
    initCounts(num_tokens, token_counts);

    // omp_countTokensTokensFirst_reduce
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        omp_countTokensTokensFirst_reduce(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "OMP Tokens First Reduce time: " << duration.count() << " seconds" << std::endl;

    // omp_countTokensElementsFirst_reduce
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        omp_countTokensElementsFirst_reduce(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "OMP Elements First Reduce time.: " << duration.count() << " seconds" << std::endl;

    // reset counts only works right if running one loop_iteration
    initCounts(num_tokens, token_counts);

    // unroll_omp_countTokensElementsFirst_reduce
    start = std::chrono::high_resolution_clock::now();
    for(int j=0; j<loop_iterations; j++) {
        unroll_omp_countTokensElementsFirst_reduce(num_elements, num_tokens, elements, tokens, token_counts);
    }
    end = std::chrono::high_resolution_clock::now();
    duration = end - start;
    std::cout << "Unroll OMP Elements First Reduce time.: " << duration.count() << " seconds" << std::endl;
}
```